[← ROS Nodes and Gazebo](../../../getting_started/ros_to_gazebo/ros_nodes_and_gazebo.rst)


# Crazyflie Multi-Sensor Fusion: From Software to Gazebo


In the [Crazyflie ROS Deployment](../python_to_ros/crazyflie_ros_deployment.ipynb) tutorial, we created a multi-sensor fusion system with **three separate sensor nodes** in software. Now we'll integrate with Gazebo's Crazyflie model, which provides its own sensor suite.

**Key Challenge**: Gazebo's sensors differ from our software simulation!

In software, we simulated:
- Motion capture (3D position)
- Barometer (altitude)
- IMU (velocity)

Gazebo's Crazyflie provides:
- Odometry (position + velocity, unified)
- IMU (acceleration + angular velocity)
- Possibly range sensors

This tutorial demonstrates **sensor fusion architecture adaptation** to work with available hardware.


## Architecture Evolution: Software → Gazebo

**Software Simulation** (previous notebook):

```
setpoint_node → /setpoint
       ↓
controller_node → /cmd_vel
       ↓
crazyflie_simulator → /mocap, /baro, /imu  ← 3 SEPARATE SENSORS
       ↓         ↓         ↓
       └─────────┴─────────┘
                 ↓
       kalman_filter → /estimate
```

**Gazebo Integration** (this notebook):

```
setpoint_node → /setpoint
       ↓
controller_node → /cmd_vel
       ↓
GAZEBO → /odom (unified sensor)  ← 1 TOPIC INSTEAD OF 3
    ↓
kalman_filter (adapted) → /estimate
```

**Architecture Adaptation**:
- Software: 3 sensor topics (mocap, baro, imu) → 7D measurement
- Gazebo: 1 sensor topic (/odom) → 6D measurement
- Must adapt KF measurement model!

:::{note}
Real robotics often requires **adapting your fusion architecture** to available sensors. This is a key skill!
:::


## What Gazebo's Crazyflie Provides

The Crazyflie model in Gazebo publishes:

**Input Topics** (subscribed by Gazebo):
- `/cmd_vel` (Twist) - Velocity commands

**Output Topics** (published by Gazebo):
- `/odom` (Odometry) - Position and velocity estimate (unified!)
- `/imu` (Imu) - Acceleration and angular velocity
- Possibly `/range` for altitude sensing

**Adaptation Strategy**:
- Extract position from `/odom` (replaces mocap + baro)
- Extract velocity from `/odom` (replaces IMU velocity)
- Simplified 1-sensor fusion (vs 3-sensor)

**Sensor Fusion Simplification**:

| Software | Gazebo |
|----------|--------|
| Mocap: 3D position | `/odom`: 3D position |
| Baro: Z altitude | `/odom`: Z component |
| IMU: 3D velocity | `/odom`: 3D velocity |
| **3 topics, 7D** | **1 topic, 6D** |


## Setup: Imports


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pykal import DynamicalSystem
from pykal.ros.ros_node import ROSNode
from pykal.algorithm_library.estimators.kf import KF
from pykal.gazebo import start_gazebo, stop_gazebo

# ROS message types
from geometry_msgs.msg import Twist, Vector3
from nav_msgs.msg import Odometry
from sensor_msgs.msg import Imu
import rclpy
import time

print("Imports successful! Ready to integrate Crazyflie with Gazebo.")

### Component Functions


In [ ]:
# ============================================================================
# Crazyflie Dynamics
# ============================================================================

def crazyflie_f(xk: np.ndarray, uk: np.ndarray, dt: float) -> np.ndarray:
    """3D constant-velocity: [x, y, z, vx, vy, vz]."""
    pos = xk[:3]
    vel = xk[3:]
    pos_new = pos + vel * dt
    vel_new = uk
    return np.vstack([pos_new, vel_new])


def h_position(xk: np.ndarray) -> np.ndarray:
    """Measure position [x, y, z]."""
    return xk[:3]


def h_velocity(xk: np.ndarray) -> np.ndarray:
    """Measure velocity [vx, vy, vz]."""
    return xk[3:]


def h_position_and_velocity(xk: np.ndarray) -> np.ndarray:
    """Measure both position and velocity (from /odom)."""
    return xk  # All 6 states


def compute_F_crazyflie(dt: float) -> np.ndarray:
    """State transition Jacobian."""
    I3 = np.eye(3)
    return np.block([
        [I3, I3 * dt],
        [np.zeros((3, 3)), I3]
    ])


def compute_H_odom() -> np.ndarray:
    """Measurement Jacobian for odometry (measures all 6 states)."""
    return np.eye(6)


# Noise covariances
Q_crazyflie = np.diag([0.001, 0.001, 0.001, 0.1, 0.1, 0.1])

# Adapted noise for Gazebo's odometry (measures position + velocity)
R_gazebo_odom = np.diag([0.01, 0.01, 0.01, 0.05, 0.05, 0.05])

print("Crazyflie dynamics defined!")
print("  State: [x, y, z, vx, vy, vz] (6D)")
print("  Gazebo odometry: measures all 6 states")

## Step 1: Launch Gazebo with Crazyflie


In [ ]:
# Launch Gazebo
print("Launching Gazebo with Crazyflie 2.1...")
print("This may take 10-20 seconds...\n")

gz = start_gazebo(
    robot='crazyflie',
    world='empty_world',
    headless=False,
    x_pose=0.0,
    y_pose=0.0,
    z_pose=0.5,  # Start at 50cm altitude
    yaw=0.0
)

print("✓ Gazebo launched successfully!")
print(f"\nGazebo Crazyflie is publishing:")
print(f"  - /odom (Odometry): Position and velocity")
print(f"  - /imu (Imu): Acceleration and angular velocity")
print(f"\nGazebo is listening on:")
print(f"  - /cmd_vel (Twist): Velocity commands")

# Give Gazebo time to initialize
import time as pytime
pytime.sleep(3)
print("\nGazebo initialization complete!")

## Step 2: Adapted Node Architecture

**Changes from software simulation**:
1. ❌ Remove `crazyflie_simulator` node (all 3 sensor publishers)
2. ❌ Remove separate mocap, baro, IMU nodes
3. ✓ Keep `setpoint_generator` (unchanged)
4. ✓ Keep `position_controller` (unchanged)
5. ✓ **Adapt** `kalman_filter` to use Gazebo's `/odom`

**Sensor Fusion Adaptation**:

```python
# Software version (3 sensors):
def filter_callback(tk, mocap, baro, imu, cmd_vel):
    yk = np.vstack([mocap, baro, imu])  # 7D
    # ...

# Gazebo version (1 sensor):
def filter_callback(tk, odom, cmd_vel):
    pos = odom[:3]
    vel = odom[7:10]
    yk = np.vstack([pos, vel])  # 6D
    # ...
```


### Node 1: Setpoint Generator (Unchanged)


In [ ]:
def create_setpoint_node(initial_position, transitions=None, rate_hz=10.0):
    """Same as before."""
    if transitions is None:
        transitions = []
    
    current_setpoint = np.array(initial_position).reshape(-1, 1)
    transition_idx = 0
    
    def setpoint_callback(tk):
        nonlocal current_setpoint, transition_idx
        
        if transition_idx < len(transitions):
            t_switch, new_pos = transitions[transition_idx]
            if tk >= t_switch:
                current_setpoint = np.array(new_pos).reshape(-1, 1)
                transition_idx += 1
        
        return {'setpoint': current_setpoint.flatten()}
    
    node = ROSNode(
        node_name='setpoint_generator',
        callback=setpoint_callback,
        subscribes_to=[],
        publishes_to=[('setpoint', Vector3, '/setpoint')],
        rate_hz=rate_hz
    )
    return node


setpoint_node = create_setpoint_node(
    initial_position=[0.0, 0.0, 1.0],
    transitions=[
        (15.0, [0.5, 0.5, 1.2]),
        (30.0, [0.0, 0.0, 1.0])
    ],
    rate_hz=10.0
)

print("✓ Setpoint generator created (unchanged)")

### Node 2: Position Controller (Unchanged)


In [ ]:
def create_position_controller_node(Kp=0.8, max_vel=0.5, rate_hz=50.0):
    """Same as before."""
    def controller_callback(tk, setpoint, estimate):
        r = setpoint
        phat = estimate[:3]
        
        error = r - phat
        v_cmd = Kp * error
        v_cmd = np.clip(v_cmd, -max_vel, max_vel)
        
        cmd_vel = np.concatenate([v_cmd, np.zeros(3)])
        return {'cmd_vel': cmd_vel}
    
    node = ROSNode(
        node_name='position_controller',
        callback=controller_callback,
        subscribes_to=[
            ('/setpoint', Vector3, 'setpoint'),
            ('/estimate', Odometry, 'estimate'),
        ],
        publishes_to=[('cmd_vel', Twist, '/cmd_vel')],
        rate_hz=rate_hz,
        required_topics={'setpoint', 'estimate'}
    )
    return node


controller_node = create_position_controller_node(Kp=0.8, max_vel=0.5, rate_hz=50.0)
print("✓ Position controller created (unchanged)")

### Node 3: Adapted Kalman Filter (Uses Gazebo's /odom)

**Key Adaptation**: Single `/odom` topic instead of 3 sensor topics.

**Measurement Model Change**:

Software:

$$
y_k = \begin{bmatrix} y_{mocap} \\ y_{baro} \\ y_{imu} \end{bmatrix} \in \mathbb{R}^7
$$

$$
H = \begin{bmatrix} I_3 & 0 \\ \begin{bmatrix}0 & 0 & 1\end{bmatrix} & 0 \\ 0 & I_3 \end{bmatrix}
$$

Gazebo:

$$
y_k = \begin{bmatrix} \vec{p} \\ \vec{v} \end{bmatrix} \in \mathbb{R}^6
$$

$$
H = I_6
$$

**Simpler fusion**, but demonstrates architecture adaptation!


In [ ]:
def create_kalman_filter_node(dt=0.01, rate_hz=50.0, Q=None, R=None):
    """
    Adapted Kalman filter for Gazebo's odometry.
    
    Now uses single /odom topic instead of 3 sensors!
    """
    if Q is None:
        Q = Q_crazyflie
    if R is None:
        R = R_gazebo_odom
    
    xhat = np.array([[0.0], [0.0], [1.0], [0.0], [0.0], [0.0]])
    P = np.diag([0.1, 0.1, 0.1, 1.0, 1.0, 1.0])
    
    def filter_callback(tk, odom, cmd_vel):
        """
        Fuse Gazebo's odometry.
        
        Args:
            tk (float): Current time
            odom (np.ndarray): (13,) Gazebo Odometry
            cmd_vel (np.ndarray): (6,) Twist command
        """
        nonlocal xhat, P
        
        # Extract position and velocity from Gazebo's Odometry
        # Odometry format: [px, py, pz, qx, qy, qz, qw, vx, vy, vz, wx, wy, wz]
        pos_meas = odom[:3]  # [x, y, z]
        vel_meas = odom[7:10]  # [vx, vy, vz]
        
        # Concatenate into measurement vector
        yk = np.vstack([pos_meas.reshape(-1, 1), vel_meas.reshape(-1, 1)])
        
        # Extract control input
        uk = cmd_vel[:3].reshape(-1, 1)
        
        # Compute Jacobians
        Fk = compute_F_crazyflie(dt)
        Hk = compute_H_odom()  # Identity matrix (measures all states)
        
        # Run KF update
        f_params = {'xk': xhat, 'uk': uk, 'dt': dt}
        h_params = {'xk': xhat}
        
        xhat_new, P_new = KF.f(
            x_P=(xhat, P),
            y=yk,
            u=uk,
            f=crazyflie_f,
            F=lambda **params: Fk,
            Q=lambda **params: Q,
            h=h_position_and_velocity,
            H=lambda **params: Hk,
            R=lambda **params: R,
            f_params=f_params,
            F_params={},
            Q_params={},
            h_params=h_params,
            H_params={},
            R_params={}
        )
        
        xhat = xhat_new
        P = P_new
        
        # Convert to Odometry format for publishing
        pos_est = xhat[:3].flatten()
        vel_est = xhat[3:].flatten()
        
        estimate = np.concatenate([
            pos_est,
            [0.0, 0.0, 0.0, 1.0],  # quaternion
            vel_est,
            [0.0, 0.0, 0.0]  # angular velocity
        ])
        
        return {'estimate': estimate}
    
    node = ROSNode(
        node_name='kalman_filter',
        callback=filter_callback,
        subscribes_to=[
            ('/odom', Odometry, 'odom'),  # ← FROM GAZEBO (single sensor!)
            ('/cmd_vel', Twist, 'cmd_vel'),
        ],
        publishes_to=[('estimate', Odometry, '/estimate')],
        rate_hz=rate_hz,
        required_topics={'odom', 'cmd_vel'}
    )
    return node


kf_node = create_kalman_filter_node(dt=0.01, rate_hz=50.0)
print("✓ Kalman filter created (ADAPTED for Gazebo!)")
print("  Software version: 3 sensors → 7D measurement")
print("  Gazebo version: 1 sensor (/odom) → 6D measurement")
print("  → Demonstrates architecture adaptation!")

## Step 3: Run the Integrated System


In [ ]:
# Initialize ROS2
rclpy.init()

# Create nodes
print("Creating ROS2 nodes...")
setpoint_node.create_node()
controller_node.create_node()
kf_node.create_node()
print("All nodes created!")

# Start nodes
print("\nStarting nodes...")
setpoint_node.start()
print("  ✓ Setpoint generator running")
kf_node.start()
print("  ✓ Kalman filter running (using Gazebo's /odom!)")
controller_node.start()
print("  ✓ Position controller running (sending /cmd_vel to Gazebo!)")

print("\n🚀 Crazyflie Gazebo integration is live!")
print("\nSystem Architecture:")
print("  setpoint_generator → /setpoint")
print("           ↓")
print("  position_controller → /cmd_vel → GAZEBO (physics + sensors)")
print("                                       ↓")
print("                                   /odom (unified sensor)")
print("                                       ↓")
print("                              kalman_filter → /estimate")
print("                                       ↓ (feedback)")
print("                              position_controller")

print("\n💡 Open Gazebo GUI to watch the Crazyflie fly!")
print("💡 Run 'rqt_graph' to see how architecture changed from software version!")

### Data Logger


In [ ]:
def create_data_logger_node():
    data_log = {
        'time': [],
        'setpoint': [],
        'cmd_vel': [],
        'odom': [],
        'estimate': []
    }
    
    def logger_callback(tk, setpoint, cmd_vel, odom, estimate):
        data_log['time'].append(tk)
        data_log['setpoint'].append(setpoint.copy())
        data_log['cmd_vel'].append(cmd_vel.copy())
        data_log['odom'].append(odom.copy())
        data_log['estimate'].append(estimate.copy())
        return {}
    
    node = ROSNode(
        node_name='data_logger',
        callback=logger_callback,
        subscribes_to=[
            ('/setpoint', Vector3, 'setpoint'),
            ('/cmd_vel', Twist, 'cmd_vel'),
            ('/odom', Odometry, 'odom'),
            ('/estimate', Odometry, 'estimate'),
        ],
        publishes_to=[],
        rate_hz=50.0
    )
    return node, data_log


logger_node, data_log = create_data_logger_node()
logger_node.create_node()
logger_node.start()
print("Data logger started!")

# Run simulation
T_sim = 45.0
print(f"\nRunning Gazebo simulation for {T_sim} seconds...")
print("Watch the Crazyflie hover and move in Gazebo!\n")
pytime.sleep(T_sim)

print(f"\nSimulation complete! Collected {len(data_log['time'])} samples.")

### Stop the System


In [ ]:
# Stop ROS nodes
print("Stopping ROS nodes...")
logger_node.stop()
setpoint_node.stop()
controller_node.stop()
kf_node.stop()
print("All nodes stopped.")

# Shutdown ROS2
rclpy.shutdown()
print("ROS2 shutdown complete.")

# Stop Gazebo
print("\nStopping Gazebo...")
stop_gazebo(gz)
print("Gazebo stopped.")

## Visualization: Architecture Adaptation Analysis


In [ ]:
# Convert to arrays
time_vec = np.array(data_log['time'])
odom_data = np.array(data_log['odom'])
estimates = np.array(data_log['estimate'])
commands = np.array(data_log['cmd_vel'])
setpoints = np.array(data_log['setpoint'])

# Extract positions
odom_pos = odom_data[:, :3]
est_pos = estimates[:, :3]
odom_vel = odom_data[:, 7:10]
est_vel = estimates[:, 7:10]

# Plotting
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(15, 10))

# Plot 1: 3D Trajectory
ax = fig.add_subplot(2, 3, 1, projection='3d')
ax.plot(odom_pos[:, 0], odom_pos[:, 1], odom_pos[:, 2],
        'gray', linewidth=1, label='Gazebo Odom', alpha=0.5)
ax.plot(est_pos[:, 0], est_pos[:, 1], est_pos[:, 2],
        'r--', linewidth=2, label='KF Estimate', alpha=0.8)
ax.scatter(setpoints[:, 0], setpoints[:, 1], setpoints[:, 2],
           c='green', s=100, marker='*', label='Setpoints')
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_zlabel('Z (m)')
ax.set_title('Gazebo Integration: 3D Trajectory', fontweight='bold')
ax.legend()

# Plot 2: XY Trajectory
ax = fig.add_subplot(2, 3, 2)
ax.plot(odom_pos[:, 0], odom_pos[:, 1], 'gray', alpha=0.5, label='Gazebo Odom')
ax.plot(est_pos[:, 0], est_pos[:, 1], 'r--', linewidth=2, label='KF Estimate')
ax.scatter(setpoints[::200, 0], setpoints[::200, 1], c='green', s=100, marker='*')
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_title('XY Trajectory (Top View)', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axis('equal')

# Plot 3: Z Altitude
ax = fig.add_subplot(2, 3, 3)
ax.plot(time_vec, odom_pos[:, 2], 'gray', label='Gazebo Odom Z', alpha=0.5)
ax.plot(time_vec, est_pos[:, 2], 'r--', linewidth=2, label='KF Estimate Z')
ax.plot(time_vec, setpoints[:, 2], 'g:', linewidth=2, label='Setpoint Z')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Z (m)')
ax.set_title('Altitude Tracking', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 4: Velocity X
ax = fig.add_subplot(2, 3, 4)
ax.plot(time_vec, odom_vel[:, 0], 'gray', label='Gazebo Vx', alpha=0.5)
ax.plot(time_vec, est_vel[:, 0], 'r--', label='KF Est. Vx')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Vx (m/s)')
ax.set_title('X Velocity: Odometry Fusion', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 5: Position Error
ax = fig.add_subplot(2, 3, 5)
pos_error = np.linalg.norm(odom_pos - est_pos, axis=1)
ax.plot(time_vec, pos_error * 1000, 'm-', linewidth=1.5)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Error (mm)')
ax.set_title('KF Correction of Gazebo Odometry', fontweight='bold')
ax.grid(True, alpha=0.3)

# Plot 6: Commands
ax = fig.add_subplot(2, 3, 6)
ax.plot(time_vec, commands[:, 0], label='Vx cmd', alpha=0.7)
ax.plot(time_vec, commands[:, 1], label='Vy cmd', alpha=0.7)
ax.plot(time_vec, commands[:, 2], label='Vz cmd', alpha=0.7)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Command (m/s)')
ax.set_title('Commands to Gazebo', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nGazebo Integration Performance:")
print(f"  Mean position error: {np.mean(pos_error)*1000:.2f} mm")
print(f"  Max position error: {np.max(pos_error)*1000:.2f} mm")

## Summary: Multi-Sensor Architecture Adaptation

We've successfully adapted our multi-sensor fusion system to work with Gazebo!

**Architecture Evolution Summary**:

| Aspect | Software | Gazebo | Change |
|--------|----------|--------|--------|
| Sensors | 3 topics | 1 topic | **Simplified** |
| Mocap | `/mocap` (Vector3) | `/odom` position | **Unified** |
| Barometer | `/baro` (Float64) | `/odom` Z | **Merged** |
| IMU | `/imu` (Vector3) | `/odom` velocity | **Merged** |
| Measurement dim | 7D stacked | 6D unified | **Reduced** |
| Noise matrix | 7×7 block-diag | 6×6 diagonal | **Simplified** |
| KF subscribes | 4 topics | 2 topics | **Fewer** |

**What Changed**:
- ❌ Removed: `crazyflie_simulator` (3 sensor publishers)
- ✓ Adapted: Kalman filter measurement model (7D → 6D)
- ✓ Unchanged: Setpoint generator, position controller

**Key Lessons**:

1. **Sensor Suite Adaptation**: Real systems rarely match ideal designs
2. **Measurement Model Flexibility**: KF can adapt to different sensors
3. **Sometimes Simpler**: Gazebo's unified `/odom` is easier than 3 sensors
4. **Code Reuse**: Controller unchanged across adaptations

**Complete Pipeline Achieved**:

```
Theory → Software → ROS → Gazebo → Hardware
  ✓         ✓        ✓       ✓         (next!)
```

**ROS Graph Evolution**:

```
Software:  /mocap, /baro, /imu → KF (3-way fusion)
Gazebo:    /odom → KF (1 sensor)
Hardware:  ??? sensors → KF (adapt again!)
```

**The Robotics Reality**: Sensor fusion architectures must adapt to:
- Available hardware sensors
- Different sensor rates
- Varying noise characteristics
- Budget and weight constraints

This notebook taught the essential skill of **architecture adaptation**. The same principles apply when deploying to real hardware: adapt to available sensors, keep control logic unchanged!


[← ROS Nodes and Gazebo](../../../getting_started/ros_to_gazebo/ros_nodes_and_gazebo.rst)
